In [15]:
import pandas as pd
import nums_from_string
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import nums_from_string

def get_links():
    ser = Service("./chromedriver")
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    links = []
    driver = webdriver.Chrome(options=options, service=ser)
    driver.get("https://badi.com/es/s/Barcelona--Espa%C3%B1a?bounds=41.4682974272428,2.22804492421789;41.31703848925413,2.052333262952554&center=41.3873974,2.168568&d=11.164609446567885&city=Barcelona&pid=ChIJ5TCOcRaYpBIRCmZHTz37sEQ&z=12")
    for i in range(2): #cada uno son 20 links
        enlaces = driver.find_elements(By.XPATH,"//a[contains(@data-qa, 'room-card-link')]")
        for a in enlaces:
            links.append(a.get_attribute('href')) 
        nb = driver.find_element(By.XPATH, "//button[contains(@class, 'sc-9fxd19-1 jEzqfk')]")
        nb.click()
        time.sleep(2)
    df = pd.DataFrame(links)
    df = df.rename(columns={0: 'urls'})
    df.to_csv('urls_badi.csv')
    driver.quit()
    print('DONE')
    return

def get_precios_Barcelona():
    inicio = time.time()
    prop_dict_list = []    
    df = pd.read_csv('urls_badi.csv')
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    driver = webdriver.Chrome('./chromedriver_mac64/chromedriver', options=options)
    for i in df['urls']:
        print(i)
        prop_dict = {}
        dato_valido = 1
        page = driver.get(i)

        try: 
            tamanho_html = driver.find_element(By.XPATH,"//p[contains(@data-qa, 'section-abstract-icon-label-area')]").text
            precio_html = driver.find_element(By.XPATH,"//h4[contains(@data-qa, 'room-details-current-price')]").text
        except:
            dato_valido=0
        
        tamanho_float = float(tamanho_html[:-2])
        precio_str = precio_html[:-4]
        precio_float = float(precio_str.replace('.', '').replace(',', '').replace('€', ''))
        if(tamanho_float > 0):
            precio_x_metro = precio_float/tamanho_float
            prop_dict['Barcelona'] = precio_x_metro
            if (dato_valido):
                prop_dict_list.append(prop_dict)
        time.sleep(1)
    df = pd.DataFrame(prop_dict_list)
    df.to_csv('Precios_Barcelona.csv')
    driver.quit()
    fin = time.time()
    print('DONE')
    print('tiempo:',(fin-inicio)//60,'minutos y',round((fin-inicio)%60) ,'segundos')
    return df

In [5]:
get_links()

DONE


In [13]:
get_precios_Barcelona()